In [1]:
from unified_planning.shortcuts import *
from unified_planning.io import PDDLReader
from VSLAM import *

from copy import deepcopy
import os
import random

### CHOOSE DOMAIN AND RATE HERE

In [2]:
domain_name = "blocks" # from blocks, driverlog, miconic, satellite
ratio_neg_pos = 10 # negative to positive ratio

Positive and negative demonstrations are generated from solution plans

In [3]:
get_environment().credits_stream = None
reader = PDDLReader()

pos_demonstrations = set()
neg_demonstrations = set()
problems = sorted(os.listdir("benchmarks/{}/problems/".format(domain_name)))
for problem_file in problems:
    print(problem_file)
    problem = reader.parse_problem('benchmarks/{}/domain.pddl'.format(domain_name), 'benchmarks/{}/problems/{}'.format(domain_name, problem_file))

    new_transitions, new_failures = generate_transitions_from_problem(problem)
    new_pos = lift_transitions(new_transitions, problem.actions)
    new_neg = lift_transitions(new_failures, problem.actions)

    pos_demonstrations.update(new_pos)
    neg_demonstrations.update(new_neg)

problem-00.pddl
SequentialPlan:
    unstack(b8, b3, b1)
    putdown(b8, b1, b2)
    unstack(b3, b4, b1)
    putdown(b3, b1, b2)
    unstack(b7, b2, b1)
    putdown(b7, b1, b2)
    unstack(b2, b6, b1)
    putdown(b2, b1, b3)
    unstack(b6, b5, b1)
    putdown(b6, b1, b2)
    unstack(b5, b1, b2)
    stack(b5, b4, b1)
    pickup(b1, b2, b3)
    stack(b1, b3, b2)
    pickup(b6, b1, b2)
    stack(b6, b5, b1)
    pickup(b2, b1, b3)
    stack(b2, b6, b1)
    pickup(b7, b1, b2)
    stack(b7, b2, b1)
From 20 transitions to 13 lifted transitions
From 80 transitions to 71 lifted transitions
problem-01.pddl
SequentialPlan:
    unstack(b5, b3, b1)
    putdown(b5, b1, b2)
    unstack(b7, b6, b1)
    stack(b7, b5, b1)
    unstack(b3, b2, b1)
    putdown(b3, b1, b2)
    unstack(b2, b1, b3)
    putdown(b2, b1, b3)
    pickup(b6, b1, b2)
    stack(b6, b3, b1)
    unstack(b7, b5, b1)
    putdown(b7, b1, b2)
    pickup(b5, b1, b2)
    stack(b5, b2, b1)
    unstack(b4, b8, b1)
    stack(b4, b1, b2)
    pi

Shuffling

In [4]:
pos_demonstrations = sorted(list(pos_demonstrations))
neg_demonstrations = sorted(list(neg_demonstrations))

random.seed(12345)
random.shuffle(pos_demonstrations)
random.shuffle(neg_demonstrations)

print("{} Demonstrations ({} positive, {} negative)".format(len(pos_demonstrations) + len(neg_demonstrations), len(pos_demonstrations), len(neg_demonstrations)))

948 Demonstrations (66 positive, 882 negative)


Preparation of the training and testing set of demonstrations using a 50/50 split

In [5]:
all_actions = problem.actions
all_fluents = problem.fluents
static_fluents = problem.get_static_fluents()

ratio_training_test = 0.5

pos_cut = int(len(pos_demonstrations)*ratio_training_test)
neg_cut = int(len(neg_demonstrations)*ratio_training_test)


training_pos_demonstrations = pos_demonstrations[:pos_cut]
testing_pos_demonstrations = pos_demonstrations[pos_cut:]
training_neg_demonstrations = neg_demonstrations[:neg_cut]
testing_neg_demonstrations = neg_demonstrations[neg_cut:]


training_neg_demonstrations = training_neg_demonstrations[:len(training_pos_demonstrations)*ratio_neg_pos]


seen_actions = {action.name:False for action in all_actions}

initial_pos_demonstrations = []
noninitial_pos_demonstrations = []

for demonstration in training_pos_demonstrations:
    action_name = demonstration[1][0]
    if not seen_actions[action_name]:
        initial_pos_demonstrations += [demonstration]
        seen_actions[action_name] = True
    else:
        noninitial_pos_demonstrations += [demonstration]


### Build demonstration set by spacing out pos and neg demonstrations
snapshots = []
slice_length = len(training_neg_demonstrations)/len(noninitial_pos_demonstrations)
demonstrations = []
for i in range(len(noninitial_pos_demonstrations)):
    start = int(slice_length * i)
    end = int(slice_length * (i+1))
    
    demonstrations += training_neg_demonstrations[start:end]
    demonstrations += noninitial_pos_demonstrations[i:i+1]
    snapshots += [end+i]

Initialization of VSLAM

In [6]:
L_pre, U_pre, L_eff, U_eff = VSLAM_initialization(all_fluents, all_actions, static_fluents)

for demonstration in initial_pos_demonstrations:
    run_VSLAM_iteration(L_pre, U_pre, L_eff, U_eff, demonstration)

Learning and evaluation

In [7]:
sound_results = []
complete_results = []
combined_results = []

for i in range(snapshots[-1]+1):
    demonstration = demonstrations[i]   
    action_name = demonstration[1][0]

    run_VSLAM_iteration(L_pre, U_pre, L_eff, U_eff, demonstration)

    if len(L_eff[action_name]) == 0 or len(U_eff[action_name]) == 0:
        print("Version space collapsed at demonstration %s" % i)
        break

    if i in snapshots:
        print("SNAPSHOT: %s" % snapshots.index(i))
        sound_f1, complete_f1 = evaluate_f1score(testing_pos_demonstrations, testing_neg_demonstrations, L_pre, U_pre, L_eff, U_eff)
        sound_results += [sound_f1]
        complete_results += [complete_f1]
        combined_results += [max(sound_f1,complete_f1)]

SNAPSHOT: 0
1 0.0 0.18655246252676635 1.0
0.0 0.31
SNAPSHOT: 1
1.0 0.18181818181818182 0.2624891926386428 1.0
0.31 0.42
SNAPSHOT: 2
1.0 0.18181818181818182 0.27837990250986827 1.0
0.31 0.44
SNAPSHOT: 3
1.0 0.18181818181818182 0.3064055334185398 1.0
0.31 0.47
SNAPSHOT: 4
1.0 0.24242424242424243 0.3794170854271354 1.0
0.39 0.55
SNAPSHOT: 5
1.0 0.30303030303030304 0.39325842696629204 1.0
0.47 0.56
SNAPSHOT: 6
1.0 0.3333333333333333 0.4078389830508474 1.0
0.5 0.58
SNAPSHOT: 7
1.0 0.3333333333333333 0.41147132169576045 1.0
0.5 0.58
SNAPSHOT: 8
1.0 0.3333333333333333 0.41479332849891204 1.0
0.5 0.59
SNAPSHOT: 9
1.0 0.45454545454545453 0.45993031358885017 1.0
0.62 0.63
SNAPSHOT: 10
1.0 0.45454545454545453 0.5344129554655871 1.0
0.62 0.7
SNAPSHOT: 11
1.0 0.6363636363636364 0.5402455661664394 1.0
0.78 0.7
SNAPSHOT: 12
1.0 0.6363636363636364 0.5689655172413793 1.0
0.78 0.73
SNAPSHOT: 13
1.0 0.6363636363636364 0.5963855421686748 1.0
0.78 0.75
SNAPSHOT: 14
1.0 0.6363636363636364 0.6470588235294118

Store the results

In [8]:
with open("benchmarks/{}/f1_ration_{}_results.csv".format(domain_name, ratio_neg_pos), "w") as f:
    for i in range(len(combined_results)):
        f.write("{}, {}, {}, {}, {}\n".format(i, snapshots[i], sound_results[i], complete_results[i], combined_results[i]))